In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox
import pyodbc
import datetime


# Configuration
db_database = 'KnowledgeBaseDB'
db_user = 'dbuser'
db_password = 'admin'
db_engine = '.\\SQLEXPRESS'

# Connection string
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'  
        f'SERVER={db_engine};'
        f'DATABASE={db_database};'
        f'UID={db_user};'
        f'PWD={db_password}'
    )
    print("Database connection successful!")
except pyodbc.Error as e:
    print("Error connecting to database:", e)


# Upload document
def upload_document():
    conn = connect_to_db()
    if not conn:
        return
    
    # File dialog to select document
    file_path = filedialog.askopenfilename(title="Select a Document")
    if not file_path:
        return

    # Get metadata
    title = title_entry.get()
    keywords = keywords_entry.get()
    summary = summary_text.get("1.0", tk.END).strip()

    if not title:
        messagebox.showerror("Input Error", "Title is required!")
        return

    # Save file in /knowledge_base/documents
    destination_folder = "./knowledge_base/documents"
    os.makedirs(destination_folder, exist_ok=True)
    file_name = os.path.basename(file_path)
    destination_path = os.path.join(destination_folder, file_name)

    try:
        # Copy file to the destination folder
        with open(file_path, "rb") as source, open(destination_path, "wb") as destination:
            destination.write(source.read())
        
        # Insert metadata into the database
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO KnowledgeBase (Title, FilePath, Keywords, UploadDate, Summary)
            VALUES (?, ?, ?, ?, ?)
        """, title, destination_path, keywords, datetime.datetime.now(), summary)
        conn.commit()
        messagebox.showinfo("Success", "Document uploaded successfully!")
    except Exception as e:
        messagebox.showerror("Upload Error", f"Error uploading document: {str(e)}")
    finally:
        conn.close()

# Search documents
def search_documents():
    conn = connect_to_db()
    if not conn:
        return

    search_query = search_entry.get().strip()
    if not search_query:
        messagebox.showerror("Input Error", "Please enter a search query!")
        return

    try:
        # Query database for matching documents
        cursor = conn.cursor()
        cursor.execute("""
            SELECT Title, FilePath, UploadDate, Summary
            FROM KnowledgeBase
            WHERE Title LIKE ? OR Keywords LIKE ?
        """, f"%{search_query}%", f"%{search_query}%")
        results = cursor.fetchall()

        # Clear previous results
        results_list.delete(0, tk.END)

        # Display results
        for row in results:
            title, file_path, upload_date, summary = row
            results_list.insert(tk.END, f"Title: {title} | Date: {upload_date.strftime('%Y-%m-%d')}")
            results_list.insert(tk.END, f"Path: {file_path}")
            results_list.insert(tk.END, f"Summary: {summary}")
            results_list.insert(tk.END, "-" * 50)
    except Exception as e:
        messagebox.showerror("Search Error", f"Error searching documents: {str(e)}")
    finally:
        conn.close()

# GUI Setup
root = tk.Tk()
root.title("Knowledge Base System")

# Upload section
tk.Label(root, text="Upload Document").grid(row=0, column=0, columnspan=2, pady=10)

tk.Label(root, text="Title:").grid(row=1, column=0, sticky="e")
title_entry = tk.Entry(root, width=50)
title_entry.grid(row=1, column=1, padx=10)

tk.Label(root, text="Keywords:").grid(row=2, column=0, sticky="e")
keywords_entry = tk.Entry(root, width=50)
keywords_entry.grid(row=2, column=1, padx=10)

tk.Label(root, text="Summary:").grid(row=3, column=0, sticky="ne")
summary_text = tk.Text(root, width=38, height=5)
summary_text.grid(row=3, column=1, padx=10, pady=5)

upload_button = tk.Button(root, text="Upload", command=upload_document)
upload_button.grid(row=4, column=1, pady=10, sticky="e")

# Search section
tk.Label(root, text="Search Documents").grid(row=5, column=0, columnspan=2, pady=10)

search_entry = tk.Entry(root, width=50)
search_entry.grid(row=6, column=0, columnspan=2, padx=10, pady=5)

search_button = tk.Button(root, text="Search", command=search_documents)
search_button.grid(row=7, column=1, pady=10, sticky="e")

# Results section
results_list = tk.Listbox(root, width=80, height=15)
results_list.grid(row=8, column=0, columnspan=2, padx=10, pady=10)

root.mainloop()
